### Import water supply data & Create supply table
Here we download the raw supply data for years 2000, 2005, and 2010 from from downscaled CMIP5 hydrology projections ([link](http://gdo-dcp.ucllnl.org/downscaled_cmip_projections/techmemo/BCSD5HydrologyMemo.pdf)).

These data include monthly estimates of runoff, precipitation, evapotranspiration, and soil moisture content at a 1/8th degree spatial resolution across the US for the period of 1950 to 2099. Estimates are provided for 21 different climate projection ensembles applied to the Variable Infiltration Capacity (VIC) Macroscale Hydrologic Model ([link](http://vic.readthedocs.io/en/master/)); see the PDF document for a complete list. For demonstration purposes, this project uses the National Center for Atmospheric Research CCSM4 2.6 projection ensembles as the base data for water supply figures. 
 
The steps involved include:

* Download monthly runoff (total_runoff), precipitation (pr), evapotranspiration (et), and soil moisture content (smc) data, in NetCDF format, from a central data repository ([link](ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_VIC_nc/ccsm4_rcp26_r1i1p1/)) for a given sample year (2000, 2005, and 2010).

* Extract the monthly data from the downloaded NetCDF files into 4-dimensional NumPy arrays (time, parameter value, latitude, longitude).

* Aggregates the time dimension (months) into annual sums, resulting in a 3-dimensional array for each parameter, i.e. a single annual value for each 1/8th degree coordinate pair. 

* Combines these 3-dimensional arrays, one for each parameter, into a single data frame listing parameter value, latitude, and longitude. 

* Spatially join state FIPS codes to the data frame, using a county shapefile stored in the data folder. 

* Summarize supply values on FIPS to create a table that can be joined to other county level data:

| YEAR | FIPS | Precip | ET | Runoff | SoilMoisture | TotalSupply | 
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 2000 | 01001 | 0 | 0 | 0 | 0 | 0 |

In [1]:
#Import libraries
import sys, os, glob, time, datetime, urllib
import numpy as np
import pandas as pd
import netCDF4

In [2]:
#Grab the FIPS data and create a dataframe from it
print "Getting record FIPS data"
fipsURL = "https://raw.githubusercontent.com/johnpfay/USWaterAccounting/VersionZero/Data/FIPS.csv"
dfFIPS = pd.read_csv(fipsURL,dtype=np.str)

Getting record FIPS data


In [5]:
#Initialize the output file and write the header line
print "Initializing the output file"
outFile = open("..\..\Scratch\HydroData.csv",'wt')
outFile.write("YEAR,LONGITUDE,LATITUDE,COFIPS,STFIPS,RUNOFF,PRECIP,ET,SME\n")

Initializing the output file


In [7]:
#Get precip data
year = 2000
baseURL2 = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_forc_nc/ccsm4_rcp26_r1i1p1/'
prURL = baseURL2 + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.pr.{}.nc".format(year)
urllib.urlretrieve(prURL,"tmpData.nc")

('tmpData.nc', <mimetools.Message instance at 0x08B43878>)

In [15]:
#Create a netCDF object
nc = netCDF4.Dataset("tmpData.nc",mode='r')

In [21]:
#Get the parameter name and its values (the last dimension in the nc object)
param_name = nc.variables.keys()[-1]
param_vals = nc.variables.values()[-1]

In [47]:
#Create dataframes of latitude and longitude values
dfLats = pd.DataFrame(nc.variables['latitude'][:])
dfLons = pd.DataFrame(nc.variables['longitude'][:])

In [68]:
#Create a dataframe of the parameter values, summed for all month records
# The columns here are longitudes and rows are latitudes
dfParam = pd.DataFrame(param_vals[:,:,:].sum(axis=0))
dfParam.columns = dfLons[0].values.tolist()
dfParam['LAT'] = dfLats[0].values.tolist()
df = pd.melt(dfParam,id_vars=['LAT'],var_name='LON',value_name=param_name)
df.dropna(inplace=True)
df.to_csv("foo.csv",index_name="OID")

In [ ]:
#Set the year to process
year = 2000
print "Processing year {}".format(year)

#Get urls for NCAR 2.6 scenario ensembles: runoff(ro), precipitation(pr), evapotranspiration(et), soil moisture (sm)
baseURL = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_VIC_nc/ccsm4_rcp26_r1i1p1/'
baseURL2 = 'ftp://gdo-dcp.ucllnl.org/pub/dcp/archive/cmip5/hydro/BCSD_mon_forc_nc/ccsm4_rcp26_r1i1p1/'
roURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.total_runoff.{}.nc".format(year)
prURL = baseURL2 + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.pr.{}.nc".format(year)
etURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.et.{}.nc".format(year)
smURL = baseURL + "conus_c5.ccsm4_rcp26_r1i1p1.monthly.smc.{}.nc".format(year)

#These lines fix an issue with slow network connections
import socket
socket.setdefaulttimeout(30)

#Loop through each file and create an annual sum array; add it to a dictionary
dataDict = {}
url = roURL
print "...downloading data from " + url
#Retrieve the data file from the ftp server
urllib.urlretrieve(url,"tmpData.nc")

#Convert to netCDF object
nc = netCDF4.Dataset("tmpData.nc",mode='r')

#Add the lats and lons array to the dictionary
dataDict["lats"] = nc.variables['latitude'][:]
dataDict["lons"] = nc.variables['longitude'][:]

#Get the parameter name and its values
param_name = nc.variables.keys()[-1]
param_vals = nc.variables.values()[-1]

#Collapse the monthly values into a single array
dataDict[param_name] = param_vals[:,:,:].sum(axis=0)

#Close the nc object
nc.close()

#Delete the nc file
os.remove("tmpData.nc")

#Update
urllib.urlcleanup()
print "....complete"

#Write array values as X,Y table
#Create lons and lats array
lons = dataDict["lons"]
lats = dataDict["lats"]

#Initialize the index to retrieve FIPS codes
idxFIPS = 0